## Custom dataset 만들어보기

- dataset generation
- package 확인 말고는 이미지 데이터 가져와야 함.
- 어차피 lenna를 쓸 것 같아서 실행은 시키지 않음.
- resize 후 이용하기 편하게 한 곳에 저장시키는 방식을 설명하셨습니다.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
# 1.3 ver부터 imread 및 imresize 함수가 misc에서 사라졌다고 한다. 그래서 1.2.0으로 고정시켰다.
import matplotlib.pyplot as plt
%matplotlib inline
cwd = os.getcwd()
print("present:", cwd)

present: c:\Github Projects\pulse-of-DL\2. 딥러닝 들어가기


CUSTOM 영역

In [ ]:
paths = ["이미지가 있을 path", "이분은 4개를 쓰셨는데", "참고로 path이기 때문에", "해당 파일 이름까지 쓰지 않음"]
categories = ["해당", "그림의", "분류", "즉 이름"]

# resize할 이미지의 크기를 의미하구요.
imgsize = [64, 64]
use_gray = 0
data_name = "custom_data"

# path 보고 불러오는 작업이구요.
for i, path in enumerate(paths):
    print(" [%d/%d] %s" % (i, len(paths), path))
# 다 가져왔다고 알리는 거구요.
print("DATA WILL BE SAVED TO \n [%s]" % (cwd + "/data/" + data_name + ".npz"))
# .npz(numpy zip)로 묶어버리시네요.


In [ ]:
def rgb2gray(rgb):
    if len(rgb.shape) is 3:
        return np.dot(rgb[..., :3], [0.299, 0.587, 0.114])
    else:
        return rgb

In [ ]:
nclass = len(paths)
valid_exts = [".jpg", ".gif", ".png", ".tga", ".jpeg"]

imgcnt = 0
for i, relpath in zip(range(nclass), paths):
    path = cwd + "/" + relpath
    flist = os.listdir(path)
    for f in flist:
        # 이미지 파일 가져오는 과정
        if os.path.splitext(f)[1].lower() not in valid_exts:
            continue
        fullpath = os.path.join(path, f)
        currimg = imread(fullpath)

        # Grayscale로의 변환 과정
        if use_gray:
            grayimg = rgb2gray(currimg)
        else:
            grayimg = currimg
        
        # 줄이고 펼치는 과정
        graysmall = imresize(grayimg, [imgsize[0], imgsize[1]]) / 255.
        grayvec = np.reshape(graysmall, (1, -1))

        # label 데이터와 함께 전체 데이터 만드는(합치는) 과정
        # path에 따르면 모두 다른 데이터였으므로 np.eye를 통해 one hot encoding을 한 듯한 효과를 낸다.
        curr_label = np.eye(nclass, nclass)[i:i + 1, :]
        if imgcnt is 0:
            totalimg = grayvec
            totallabel = curr_label
        else:
            totalimg = np.concatenate((totalimg, grayvec), axis=0)
            totallabel = np.concatenate((totallabel, curr_label), axis=0)
        imgcnt += 1
print("Total images:" imgcnt)

In [ ]:
def print_shape(string, x):
    print("Shape of [%s] is [%s]" (string, x.shape))

In [ ]:
randidx = np.random.randint(imgcnt, size=imgcnt)
trainidx = randidx(0:int(4 * imgcnt / 5))
testidx = randidx(int(4 * imgcnt / 5):imgcnt)

trainimg = totalimg[trainidx, :]
trainlabel = totallabel[trainidx, :]
testimg = totalimg[testidx, :]
testlabel = totallabel[testidx, :]

print_shape("trainimg", trainimg)
print_shape("trainlabel", trainlabel)
print_shape("testimg", testimg)
print_shape("testlabel", testlabel)

In [ ]:
savepath = swd + "/data/" + data_name + ".npz"
np.saves(savepath, trainimg=trainimg, trainlabel=trainlabel,
         testimg=testimg, testlabel=testlabel,
         imgsize=imgsize, use_gray=use_gray, categories=categories)

In [ ]:
cwd = os.getcwd()
loadpath = cwd + "/data/" + data_name + ".npz"
l = np.load(loadpath)
print(l.files)

In [ ]:
trainimg_loaded = l["trainimg"]
trainlabel_loaded = l["trainlabel"]
testimg_loaded = l["testimg"]
testlabel_loaded = l["testlabel"]
# print로 shape을 확인하지만, [1, 64 * 64 * 3]일 테지.

In [ ]:
ntrain_loaded = trainimg_loaded.shape[0]
batch_size = 5
randidx = np.random.randint(ntrain_loaded, size=batch_size)

for i in randidx:
    currimg = np.reshape(trainimg_loaded[i, :], (imgsize[0], -1))
    currlabel_onehot = trainlabel_loaded[i, :]
    currlabel = np.argmax(currlabel_onehot)
    if use_gray:
        currimg = np.reshape(trainimg[i, :], (imgsize[0], -1))
        plt.matshow(currimg, cmap=plt.get_cmap("gray"))
        plt.colorbar()
    else:
        currimg = np.reshape(trainimg[i, :], (imgsize[0], imgsize[1], 3))
        plt.imshow(currimg)
    title_string = ("[%d] CLASS-%d (%s)"
                    % (i, currlabel, categories_loaded[currlabel]))
    plt.title(title_string)
    plt.show()